# Define input data

In [13]:
run = 505637
datadir = f"/eos/user/j/jpotgiet/SWAN_projects/Raw_data/{run}"

# Set up PyROOT environment and load digits file

In [14]:
import ROOT

%jsroot on

In [20]:
import pathlib

datadir = pathlib.Path(datadir)

digfile = ROOT.TFile(str(datadir / "trddigits.root"))
digtree = digfile.Get("o2sim")

digtree.Print()

******************************************************************************
*Tree    :o2sim     : o2sim                                                  *
*Entries :      198 : Total =       240985791 bytes  File  Size =   59832865 *
*        :          : Tree compression factor =   4.03                       *
******************************************************************************
*Br    0 :TRDDigit  : Int_t TRDDigit_                                        *
*Entries :      198 : Total  Size=       9894 bytes  File Size  =        881 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=   1.89     *
*............................................................................*
*Br    1 :TRDDigit.mDetector : UShort_t mDetector[TRDDigit_]                 *
*Entries :      198 : Total  Size=    7400135 bytes  File Size  =      85131 *
*Baskets :       22 : Basket Size=      32000 bytes  Compression=  86.92     *
*...................................................

# Basic Checks

## Channel numbers
The channel numbers should be (almost) evenly distributed between 0 and 20. The pads at the edge of the chamber might be disabled, so it would be understanable if the outermost MCMs have the lowest or highest channels disabled.

For run 504428, the channel distribution looks very different. There seems to be something broken in generating the digits.

For run 504419, the channel distribution looks ok, but there are a few channel numbers greater than 20, and there are less digits from channels above 10 or so.

In [16]:
cnv = ROOT.TCanvas()
cnv.SetLogy()
digtree.Draw("mChannel")
cnv.Draw()

## Digits map
As the channels are somewhat weird, we should probably check if the digits are evenly distributed over chambers, ROBs and MCMs. Let's start with the chamber distribution. Some differences are expected, as parts of the detector are disabled, and the stack-2 chambers have only 6 ROBs instead of the 8 for chambers in stacks 0,1,3 and 4. 

In [7]:
cnv_det = ROOT.TCanvas("cnv_det")
digtree.Draw("mDetector>>hdet(540,-0.5,539.5)")
cnv_det.Draw()

For 504428, that doesn't look right at all. Why would detector be between 0 and 31 - so it's not just the number within the supermodule. I'm not even sure if it makes sense to look at the MCM position within the chamber, but here we go.

For 504419, the distribution looks ok, although I would have expected an even more uniform distribution.

In [8]:
cnv_map = ROOT.TCanvas("cnv_map")

digtree.Draw("mROB:mMCM>>hmap","1","colz")
# digtree.Draw("mROB:mMCM>>hmap","mDetector==4","colz")

# ROOT.hmap.Scale(1/54.) # I think I've seen 54 trigger records in this file
ROOT.hmap.Draw("text,same")
ROOT.hmap.SetStats(0)
cnv_map.Draw()

Again, the distribution is not as uniform as expected. Especially for 504428 that's strange, because the chambers were so different.

## Load tracklets file

In [21]:
trackfile = ROOT.TFile(str(datadir / "trdtracklets.root"))
tracktree = trackfile.Get("o2sim")

tracktree.Print()

******************************************************************************
*Tree    :o2sim     : o2sim                                                  *
*Entries :      198 : Total =        12843294 bytes  File  Size =    5842221 *
*        :          : Tree compression factor =   2.20                       *
******************************************************************************
*Br    0 :Tracklet  : Int_t Tracklet_                                        *
*Entries :      198 : Total  Size=       7106 bytes  File Size  =        801 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=   2.08     *
*............................................................................*
*Br    1 :Tracklet.mtrackletWord : ULong_t mtrackletWord[Tracklet_]          *
*Entries :      198 : Total  Size=   12275399 bytes  File Size  =    5715664 *
*Baskets :      198 : Basket Size=      32000 bytes  Compression=   2.15     *
*...................................................

In [28]:
# look into tracklet word
trackletWord = tracktree.GetBranch("Tracklet.mtrackletWord")

trackletWord.Print()

*Br    9 :Tracklet.mtrackletWord : ULong_t mtrackletWord[Tracklet_]          *
*Entries :      198 : Total  Size=   12275399 bytes  File Size  =    5715664 *
*Baskets :      198 : Basket Size=      32000 bytes  Compression=   2.15     *
*............................................................................*


In [29]:
# get charge values from word
trackletWord.GetEntry(0)

61668